In [1]:
import cv2
import dlib
import face_recognition
import numpy as np
import pyrealsense2 as rs

# Load pre-trained models
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks (2).dat")
face_recognizer = dlib.face_recognition_model_v1("dlib_face_recognition_resnet_model_v1.dat")

# Configure the RealSense camera
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)

# Start streaming
pipeline.start(config)

# Function to get person's geo coordinates based on camera's location
def get_person_geotags(color_frame, depth_frame, face_rect):
    # Calculate depth information for the center of the face bounding box
    depth_center = depth_frame.get_distance(int(face_rect[0] + face_rect[2] / 2), int(face_rect[1] + face_rect[3] / 2))

    # Get the exact 3D coordinates in meters
    depth_intrin = depth_frame.profile.as_video_stream_profile().intrinsics
    depth_point = rs.rs2_deproject_pixel_to_point(depth_intrin, [int(face_rect[0] + face_rect[2] / 2), int(face_rect[1] + face_rect[3] / 2)], depth_center)

    return depth_point

# Load a sample image to search for
search_image = face_recognition.load_image_file("C:\\Users\\saini\\Downloads\\WhatsApp Image 2024-02-05 at 9.17.11 PM.jpeg")
search_face_descriptor = face_recognition.face_encodings(search_image)[0]

while True:
    frames = pipeline.wait_for_frames()
    color_frame = frames.get_color_frame()
    depth_frame = frames.get_depth_frame()

    if not color_frame or not depth_frame:
        continue

    color_image = np.asanyarray(color_frame.get_data())
    depth_image = np.asanyarray(depth_frame.get_data())

    faces = detector(color_image)

    for face in faces:
        shape = predictor(color_image, face)
        face_descriptor_dlib = face_recognizer.compute_face_descriptor(color_image, shape)
        face_descriptor_numpy = np.array(face_descriptor_dlib)

        # Get 2D bounding box coordinates
        (x, y, w, h) = (face.left(), face.top(), face.width(), face.height())

        # Get person's geo coordinates based on camera's location
        person_geotags = get_person_geotags(color_frame, depth_frame, (x, y, w, h))

        # Compare the face descriptor with the search image
        match = face_recognition.compare_faces([search_face_descriptor], face_descriptor_numpy, tolerance=0.6)[0]

        if match and person_geotags:
            # Draw bounding box and text on the color image
            cv2.rectangle(color_image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(color_image, f"Lat: {29.891293 + person_geotags[0]:.6f}, Long: {77.96101021 + person_geotags[1]:.6f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # Print or use the person's geo coordinates
            print(f"Person Geotags: Latitude {29.891293 + person_geotags[0]:.6f}, Longitude {77.96101021 + person_geotags[1]:.6f}")

    # Display the color image with bounding boxes and coordinates
    cv2.imshow("RealSense Camera Search", color_image)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Stop streaming and close all windows
pipeline.stop()
cv2.destroyAllWindows()


Person Geotags: Latitude 30.012599, Longitude 77.831662
Person Geotags: Latitude 30.010080, Longitude 77.834348
Person Geotags: Latitude 30.029581, Longitude 77.849846
Person Geotags: Latitude 30.028249, Longitude 77.850916
Person Geotags: Latitude 30.012405, Longitude 77.831869
Person Geotags: Latitude 29.988674, Longitude 77.812872
Person Geotags: Latitude 29.946342, Longitude 77.779487
Person Geotags: Latitude 29.930738, Longitude 77.794030
Person Geotags: Latitude 29.930380, Longitude 77.795546
Person Geotags: Latitude 29.930380, Longitude 77.795546
Person Geotags: Latitude 29.930431, Longitude 77.795329
Person Geotags: Latitude 29.930329, Longitude 77.795763
Person Geotags: Latitude 29.949032, Longitude 77.789049
Person Geotags: Latitude 29.948960, Longitude 77.789265
Person Geotags: Latitude 29.948887, Longitude 77.789482
Person Geotags: Latitude 29.948742, Longitude 77.789915
Person Geotags: Latitude 29.948451, Longitude 77.790781
Person Geotags: Latitude 29.948305, Longitude 77

In [3]:
!pip install opencv-python-headless

Defaulting to user installation because normal site-packages is not writeable
  Using cached opencv_python_headless-4.9.0.80-cp37-abi3-win_amd64.whl (38.5 MB)


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\saini\\AppData\\Roaming\\Python\\Python39\\site-packages\\cv2\\cv2.pyd'
Check the permissions.



In [1]:
import cv2
import dlib
import face_recognition
import numpy as np
import pyrealsense2 as rs
import ctypes

# Get screen resolution
user32 = ctypes.windll.user32
screen_width = user32.GetSystemMetrics(0)
screen_height = user32.GetSystemMetrics(1)

# Load pre-trained models
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks (2).dat")
face_recognizer = dlib.face_recognition_model_v1("dlib_face_recognition_resnet_model_v1.dat")

# Configure the RealSense camera
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)

# Start streaming
pipeline.start(config)

# Function to get person's geo coordinates based on camera's location
def get_person_geotags(color_frame, depth_frame, face_rect):
    # Calculate depth information for the center of the face bounding box
    depth_center = depth_frame.get_distance(int(face_rect[0] + face_rect[2] / 2), int(face_rect[1] + face_rect[3] / 2))

    # Get the exact 3D coordinates in meters
    depth_intrin = depth_frame.profile.as_video_stream_profile().intrinsics
    depth_point = rs.rs2_deproject_pixel_to_point(depth_intrin, [int(face_rect[0] + face_rect[2] / 2), int(face_rect[1] + face_rect[3] / 2)], depth_center)

    return depth_point

# Load a sample image to search for
search_image = face_recognition.load_image_file("C:\\Users\\saini\\OneDrive\\Pictures\\Screenshots\\Screenshot 2024-02-10 103328.png")
search_face_descriptor = face_recognition.face_encodings(search_image)[0]

while True:
    frames = pipeline.wait_for_frames()
    color_frame = frames.get_color_frame()
    depth_frame = frames.get_depth_frame()

    if not color_frame or not depth_frame:
        continue

    color_image = np.asanyarray(color_frame.get_data())
    depth_image = np.asanyarray(depth_frame.get_data())

    faces = detector(color_image)

    for face in faces:
        shape = predictor(color_image, face)
        face_descriptor_dlib = face_recognizer.compute_face_descriptor(color_image, shape)
        face_descriptor_numpy = np.array(face_descriptor_dlib)

        # Get 2D bounding box coordinates
        (x, y, w, h) = (face.left(), face.top(), face.width(), face.height())

        # Get person's geo coordinates based on camera's location
        person_geotags = get_person_geotags(color_frame, depth_frame, (x, y, w, h))

        # Compare the face descriptor with the search image
        match = face_recognition.compare_faces([search_face_descriptor], face_descriptor_numpy, tolerance=0.6)[0]

        if match and person_geotags:
            # Draw bounding box and text on the color image
            cv2.rectangle(color_image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(color_image, f"Lat: {29.891293 + person_geotags[0]:.6f}, Long: {77.96101021 + person_geotags[1]:.6f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # Print or use the person's geo coordinates
            print(f"Person Geotags: Latitude {29.891293 + person_geotags[0]:.6f}, Longitude {77.96101021 + person_geotags[1]:.6f}")

    # Display the color image with bounding boxes and coordinates
    cv2.namedWindow("RealSense Camera Search", cv2.WINDOW_NORMAL)
    cv2.setWindowProperty("RealSense Camera Search", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
    cv2.imshow("RealSense Camera Search", color_image)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Stop streaming and close all windows
pipeline.stop()
cv2.destroyAllWindows()


Person Geotags: Latitude 29.891293, Longitude 77.961010
Person Geotags: Latitude 29.319074, Longitude 77.759298
Person Geotags: Latitude 29.527452, Longitude 77.757080
Person Geotags: Latitude 29.607457, Longitude 77.758467
Person Geotags: Latitude 29.589319, Longitude 77.739995
Person Geotags: Latitude 29.553054, Longitude 77.741514
Person Geotags: Latitude 29.515234, Longitude 77.760962
Person Geotags: Latitude 29.492803, Longitude 77.759298
Person Geotags: Latitude 29.490064, Longitude 77.757912
Person Geotags: Latitude 29.494446, Longitude 77.760130
Person Geotags: Latitude 29.475797, Longitude 77.760407
Person Geotags: Latitude 29.461707, Longitude 77.762764
Person Geotags: Latitude 29.515755, Longitude 77.761239
Person Geotags: Latitude 29.626478, Longitude 77.777599
Person Geotags: Latitude 29.682588, Longitude 77.775718
Person Geotags: Latitude 29.680046, Longitude 77.773462
Person Geotags: Latitude 29.662052, Longitude 77.774966
Person Geotags: Latitude 29.628288, Longitude 77

Person Geotags: Latitude 29.891293, Longitude 77.961010
Person Geotags: Latitude 29.891293, Longitude 77.961010
Person Geotags: Latitude 29.338101, Longitude 77.785574
Person Geotags: Latitude 29.541303, Longitude 77.746071
Person Geotags: Latitude 29.751249, Longitude 77.732914
Person Geotags: Latitude 29.775300, Longitude 77.726191
Person Geotags: Latitude 29.791112, Longitude 77.722266
Person Geotags: Latitude 29.807816, Longitude 77.719232
Person Geotags: Latitude 29.773185, Longitude 77.721909
Person Geotags: Latitude 29.755914, Longitude 77.722801
Person Geotags: Latitude 29.891293, Longitude 77.961010
Person Geotags: Latitude 29.821538, Longitude 78.067267
Person Geotags: Latitude 29.673329, Longitude 78.027471
Person Geotags: Latitude 29.652094, Longitude 78.047096
Person Geotags: Latitude 29.582161, Longitude 78.082674
Person Geotags: Latitude 29.471921, Longitude 77.841560
Person Geotags: Latitude 29.441419, Longitude 77.825016
Person Geotags: Latitude 29.459047, Longitude 77

Person Geotags: Latitude 30.072105, Longitude 78.022997
Person Geotags: Latitude 30.072792, Longitude 78.040758
Person Geotags: Latitude 30.071968, Longitude 78.040395
Person Geotags: Latitude 30.070594, Longitude 78.039792
Person Geotags: Latitude 30.071556, Longitude 78.040214
Person Geotags: Latitude 30.054150, Longitude 78.022808
Person Geotags: Latitude 30.071693, Longitude 78.022855
Person Geotags: Latitude 30.088209, Longitude 78.022573
Person Geotags: Latitude 30.067846, Longitude 78.004488
Person Geotags: Latitude 30.082785, Longitude 78.004015
Person Geotags: Latitude 30.083387, Longitude 78.004150
Person Geotags: Latitude 30.085045, Longitude 78.004522
Person Geotags: Latitude 30.083387, Longitude 78.021065
Person Geotags: Latitude 30.085798, Longitude 78.021819
Person Geotags: Latitude 30.084141, Longitude 78.021301
Person Geotags: Latitude 30.084141, Longitude 78.004319
Person Geotags: Latitude 30.101122, Longitude 78.004319
Person Geotags: Latitude 30.100794, Longitude 78

Person Geotags: Latitude 30.154729, Longitude 77.973519
Person Geotags: Latitude 30.154953, Longitude 77.973529
Person Geotags: Latitude 30.148217, Longitude 77.969655
Person Geotags: Latitude 30.133324, Longitude 77.969684
Person Geotags: Latitude 30.138621, Longitude 77.985980
Person Geotags: Latitude 30.125943, Longitude 77.986350
Person Geotags: Latitude 30.096860, Longitude 78.003440
Person Geotags: Latitude 30.082785, Longitude 78.020877
Person Geotags: Latitude 30.085346, Longitude 78.021678
Person Geotags: Latitude 30.085045, Longitude 78.021583
Person Geotags: Latitude 30.100630, Longitude 78.021160
Person Geotags: Latitude 30.100630, Longitude 78.021160
Person Geotags: Latitude 30.101450, Longitude 78.021395
Person Geotags: Latitude 30.083839, Longitude 78.021207
Person Geotags: Latitude 30.082935, Longitude 78.020924
Person Geotags: Latitude 30.082333, Longitude 78.020736
Person Geotags: Latitude 30.097680, Longitude 78.003609
Person Geotags: Latitude 30.109953, Longitude 78

Person Geotags: Latitude 29.689930, Longitude 78.066015
Person Geotags: Latitude 29.710768, Longitude 78.083630
Person Geotags: Latitude 29.891293, Longitude 77.961010
Person Geotags: Latitude 29.707825, Longitude 78.066604
Person Geotags: Latitude 29.688236, Longitude 78.066898
Person Geotags: Latitude 29.891293, Longitude 77.961010
Person Geotags: Latitude 29.891293, Longitude 77.961010
Person Geotags: Latitude 29.891293, Longitude 77.961010
Person Geotags: Latitude 29.891293, Longitude 77.961010
Person Geotags: Latitude 29.881017, Longitude 77.982585
Person Geotags: Latitude 29.891293, Longitude 77.961010
Person Geotags: Latitude 29.891293, Longitude 77.961010
Person Geotags: Latitude 29.891293, Longitude 77.961010
Person Geotags: Latitude 29.879894, Longitude 77.980630
Person Geotags: Latitude 29.891293, Longitude 77.961010
Person Geotags: Latitude 29.891293, Longitude 77.961010
Person Geotags: Latitude 29.891293, Longitude 77.961010
Person Geotags: Latitude 29.891293, Longitude 77

...

Person Geotags: Latitude 30.445904, Longitude 78.056148
Person Geotags: Latitude 30.379975, Longitude 78.056663
Person Geotags: Latitude 30.330292, Longitude 78.074418
Person Geotags: Latitude 30.329956, Longitude 78.074331
Person Geotags: Latitude 30.382984, Longitude 78.057252
Person Geotags: Latitude 30.469683, Longitude 78.039912
Person Geotags: Latitude 30.491125, Longitude 78.040456
Person Geotags: Latitude 30.492948, Longitude 78.040697
Person Geotags: Latitude 30.493404, Longitude 78.040758
Person Geotags: Latitude 30.489302, Longitude 78.040214
Person Geotags: Latitude 30.487479, Longitude 78.039973
Person Geotags: Latitude 30.489757, Longitude 78.040275
Person Geotags: Latitude 30.397945, Longitude 78.082326
Person Geotags: Latitude 30.453723, Longitude 78.078502
Person Geotags: Latitude 30.476763, Longitude 78.058430
Person Geotags: Latitude 30.473223, Longitude 78.040395
Person Geotags: Latitude 30.487479, Longitude 78.039973
Person Geotags: Latitude 30.473666, Longitude 78

Person Geotags: Latitude 30.332988, Longitude 77.993816
Person Geotags: Latitude 30.333355, Longitude 77.993843
Person Geotags: Latitude 30.330791, Longitude 77.993652
Person Geotags: Latitude 30.343860, Longitude 77.985754
Person Geotags: Latitude 30.344988, Longitude 77.985816
Person Geotags: Latitude 30.329351, Longitude 77.985836
Person Geotags: Latitude 30.329351, Longitude 77.985836
Person Geotags: Latitude 30.328625, Longitude 77.985795
Person Geotags: Latitude 30.334087, Longitude 77.993897
Person Geotags: Latitude 30.334087, Longitude 77.993897
Person Geotags: Latitude 30.340474, Longitude 78.001409
Person Geotags: Latitude 30.343483, Longitude 78.001680
Person Geotags: Latitude 30.341979, Longitude 78.001545
Person Geotags: Latitude 30.331523, Longitude 77.993707
Person Geotags: Latitude 30.332622, Longitude 77.993788
Person Geotags: Latitude 30.330425, Longitude 77.993625
Person Geotags: Latitude 30.343860, Longitude 77.985754
Person Geotags: Latitude 30.334820, Longitude 77

Person Geotags: Latitude 29.521217, Longitude 78.262305
Person Geotags: Latitude 29.525413, Longitude 78.258890
Person Geotags: Latitude 29.596792, Longitude 78.231911
Person Geotags: Latitude 29.392547, Longitude 78.204657
Person Geotags: Latitude 29.425357, Longitude 78.198680
Person Geotags: Latitude 29.632358, Longitude 77.812708
Person Geotags: Latitude 29.615052, Longitude 77.858408
Person Geotags: Latitude 29.618777, Longitude 77.886992
Person Geotags: Latitude 29.601526, Longitude 77.859011
Person Geotags: Latitude 29.780031, Longitude 78.109862
Person Geotags: Latitude 29.879431, Longitude 78.026435
Person Geotags: Latitude 29.877182, Longitude 77.779644
Person Geotags: Latitude 30.004015, Longitude 77.923474
Person Geotags: Latitude 29.841063, Longitude 77.886785
Person Geotags: Latitude 29.875710, Longitude 77.857806
Person Geotags: Latitude 29.940301, Longitude 77.828688
Person Geotags: Latitude 29.937319, Longitude 77.834026
Person Geotags: Latitude 29.936745, Longitude 77

Person Geotags: Latitude 30.253956, Longitude 78.004319
Person Geotags: Latitude 30.257923, Longitude 78.004793
Person Geotags: Latitude 30.268565, Longitude 78.004049
Person Geotags: Latitude 30.255373, Longitude 78.021536
Person Geotags: Latitude 30.245887, Longitude 78.022855
Person Geotags: Latitude 30.252256, Longitude 78.054822
Person Geotags: Latitude 30.259623, Longitude 78.022243
Person Geotags: Latitude 30.250839, Longitude 78.020783
Person Geotags: Latitude 30.236434, Longitude 78.021207
Person Geotags: Latitude 30.221534, Longitude 78.021583
Person Geotags: Latitude 30.215188, Longitude 78.023656
Person Geotags: Latitude 30.195240, Longitude 78.040697
Person Geotags: Latitude 30.197773, Longitude 78.041361
Person Geotags: Latitude 30.197773, Longitude 78.041361
Person Geotags: Latitude 30.212266, Longitude 78.040576
Person Geotags: Latitude 30.218841, Longitude 78.024362
Person Geotags: Latitude 30.223955, Longitude 78.025352
Person Geotags: Latitude 30.216406, Longitude 78

RuntimeError: Device disconnected. Failed to reconnect: No device connected5000

In [1]:
!pip install opencv-python-headless

  Using cached opencv_python_headless-4.9.0.80-cp37-abi3-win_amd64.whl (38.5 MB)


In [2]:
import cv2


In [3]:
print(cv2.getBuildInformation())



General configuration for OpenCV 4.9.0 =====================================
  Version control:               4.9.0

  Platform:
    Timestamp:                   2023-12-31T11:21:12Z
    Host:                        Windows 10.0.17763 AMD64
    CMake:                       3.24.2
    CMake generator:             Visual Studio 14 2015
    CMake build tool:            MSBuild.exe
    MSVC:                        1900
    Configuration:               Debug Release

  CPU/HW features:
    Baseline:                    SSE SSE2 SSE3
      requested:                 SSE3
    Dispatched code generation:  SSE4_1 SSE4_2 FP16 AVX AVX2
      requested:                 SSE4_1 SSE4_2 AVX FP16 AVX2 AVX512_SKX
      SSE4_1 (16 files):         + SSSE3 SSE4_1
      SSE4_2 (1 files):          + SSSE3 SSE4_1 POPCNT SSE4_2
      FP16 (0 files):            + SSSE3 SSE4_1 POPCNT SSE4_2 FP16 AVX
      AVX (8 files):             + SSSE3 SSE4_1 POPCNT SSE4_2 AVX
      AVX2 (36 files):           + SSSE3 SSE4_1 

In [4]:
# Install necessary packages (assuming you have a GPU with CUDA)
!pip install opencv-python-headless opencv-contrib-python-headless

# Check OpenCV version
import cv2
print(cv2.__version__)

# Check if OpenCV is built with CUDA support
print(cv2.cuda.getCudaEnabledDeviceCount())


     ---------------------------------------- 45.2/45.2 MB 1.1 MB/s eta 0:00:00
4.9.0
0


In [5]:
import cv2

# Print OpenCV version
print("OpenCV version:", cv2.__version__)

# Check if CUDA is available
print("CUDA available:", cv2.cuda.getCudaEnabledDeviceCount() > 0)

# Print GPU information
if cv2.cuda.getCudaEnabledDeviceCount() > 0:
    for i in range(cv2.cuda.getCudaEnabledDeviceCount()):
        print("GPU {}: {}".format(i, cv2.cuda.getDeviceInfo(i)))


OpenCV version: 4.9.0
CUDA available: False


In [ ]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import cv2
import dlib
import face_recognition
import numpy as np
import pyrealsense2 as rs

# Load pre-trained models
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks (2).dat")
face_recognizer = dlib.face_recognition_model_v1("dlib_face_recognition_resnet_model_v1.dat")

# Configure the RealSense camera
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)

# Start streaming
pipeline.start(config)

# Function to get person's geo coordinates based on camera's location
def get_person_geotags(color_frame, depth_frame, face_rect):
    # Calculate depth information for the center of the face bounding box
    depth_center = depth_frame.get_distance(int(face_rect[0] + face_rect[2] / 2), int(face_rect[1] + face_rect[3] / 2))

    # Get the exact 3D coordinates in meters
    depth_intrin = depth_frame.profile.as_video_stream_profile().intrinsics
    depth_point = rs.rs2_deproject_pixel_to_point(depth_intrin, [int(face_rect[0] + face_rect[2] / 2), int(face_rect[1] + face_rect[3] / 2)], depth_center)

    return depth_point

# Load a sample image to search for
search_image = face_recognition.load_image_file("C:\\Users\\saini\\Downloads\\WhatsApp Image 2024-02-05 at 9.17.11 PM.jpeg")
search_face_descriptor = face_recognition.face_encodings(search_image)[0]

# Create Tkinter GUI
class App:
    def __init__(self, window):
        self.window = window
        self.window.title("Face Recognition App")

        self.btn_browse = tk.Button(window, text="Browse Image", command=self.browse_image)
        self.btn_browse.pack()

        self.label_image = tk.Label(window)
        self.label_image.pack()

        self.btn_start = tk.Button(window, text="Start Recognition", command=self.start_recognition)
        self.btn_start.pack()

        self.btn_stop = tk.Button(window, text="Stop Recognition", command=self.stop_recognition)
        self.btn_stop.pack()

    def browse_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg *.jpeg *.png")])
        if file_path:
            self.image = Image.open(file_path)
            self.image = self.image.resize((400, 300), Image.ANTIALIAS)
            self.photo = ImageTk.PhotoImage(self.image)
            self.label_image.config(image=self.photo)

    def start_recognition(self):
        # Start recognition process
        while True:
            frames = pipeline.wait_for_frames()
            color_frame = frames.get_color_frame()
            depth_frame = frames.get_depth_frame()

            if not color_frame or not depth_frame:
                continue

            color_image = np.asanyarray(color_frame.get_data())
            depth_image = np.asanyarray(depth_frame.get_data())

            faces = detector(color_image)

            for face in faces:
                shape = predictor(color_image, face)
                face_descriptor_dlib = face_recognizer.compute_face_descriptor(color_image, shape)
                face_descriptor_numpy = np.array(face_descriptor_dlib)

                # Get 2D bounding box coordinates
                (x, y, w, h) = (face.left(), face.top(), face.width(), face.height())

                # Get person's geo coordinates based on camera's location
                person_geotags = get_person_geotags(color_frame, depth_frame, (x, y, w, h))

                # Compare the face descriptor with the search image
                match = face_recognition.compare_faces([search_face_descriptor], face_descriptor_numpy, tolerance=0.6)[0]

                if match and person_geotags:
                    # Draw bounding box and text on the color image
                    cv2.rectangle(color_image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    cv2.putText(color_image, f"Lat: {29.891293 + person_geotags[0]:.6f}, Long: {77.96101021 + person_geotags[1]:.6f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                    # Print or use the person's geo coordinates
                    print(f"Person Geotags: Latitude {29.891293 + person_geotags[0]:.6f}, Longitude {77.96101021 + person_geotags[1]:.6f}")

            # Display the color image with bounding boxes and coordinates
            cv2.imshow("RealSense Camera Search", color_image)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    def stop_recognition(self):
        pipeline.stop()
        cv2.destroyAllWindows()

# Create an instance of the Tkinter window
root = tk.Tk()
app = App(root)
root.mainloop()


C:\Users\saini\AppData\Local\Temp\ipykernel_22540\2558371690.py:61: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  self.image = self.image.resize((400, 300), Image.ANTIALIAS)


Person Geotags: Latitude 29.959030, Longitude 77.803677
Person Geotags: Latitude 29.960043, Longitude 77.775993
Person Geotags: Latitude 29.941721, Longitude 77.779930
Person Geotags: Latitude 29.843937, Longitude 77.785077
Person Geotags: Latitude 29.891293, Longitude 77.961010
Person Geotags: Latitude 29.924750, Longitude 77.780535
Person Geotags: Latitude 29.959369, Longitude 77.782978
Person Geotags: Latitude 29.961956, Longitude 77.796881
Person Geotags: Latitude 29.958724, Longitude 77.784665
Person Geotags: Latitude 29.962856, Longitude 77.794791
Person Geotags: Latitude 29.942480, Longitude 77.777204
Person Geotags: Latitude 29.941890, Longitude 77.779324
Person Geotags: Latitude 29.941890, Longitude 77.779324
Person Geotags: Latitude 29.957003, Longitude 77.789165
Person Geotags: Latitude 29.960493, Longitude 77.800279
Person Geotags: Latitude 29.976950, Longitude 77.817384
Person Geotags: Latitude 30.058695, Longitude 77.819949
Person Geotags: Latitude 30.081239, Longitude 77

In [ ]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import cv2
import dlib
import face_recognition
import numpy as np
import pyrealsense2 as rs

# Load pre-trained models
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks (2).dat")
face_recognizer = dlib.face_recognition_model_v1("dlib_face_recognition_resnet_model_v1.dat")

# Configure the RealSense camera
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)

# Start streaming
pipeline.start(config)

# Function to get person's geo coordinates based on camera's location
def get_person_geotags(color_frame, depth_frame, face_rect):
    # Calculate depth information for the center of the face bounding box
    depth_center = depth_frame.get_distance(int(face_rect[0] + face_rect[2] / 2), int(face_rect[1] + face_rect[3] / 2))

    # Get the exact 3D coordinates in meters
    depth_intrin = depth_frame.profile.as_video_stream_profile().intrinsics
    depth_point = rs.rs2_deproject_pixel_to_point(depth_intrin, [int(face_rect[0] + face_rect[2] / 2), int(face_rect[1] + face_rect[3] / 2)], depth_center)

    return depth_point

# Load a sample image to search for
search_image = face_recognition.load_image_file("C:\\Users\\saini\\Downloads\\WhatsApp Image 2024-02-05 at 9.17.11 PM.jpeg")
search_face_descriptor = face_recognition.face_encodings(search_image)[0]

# Create Tkinter GUI
class App:
    def __init__(self, window):
        self.window = window
        self.window.title("Face Recognition App")

        self.btn_browse = tk.Button(window, text="Browse Image", command=self.browse_image)
        self.btn_browse.pack()

        self.label_image = tk.Label(window)
        self.label_image.pack()

        self.btn_start = tk.Button(window, text="Start Recognition", command=self.start_recognition)
        self.btn_start.pack()

        self.btn_stop = tk.Button(window, text="Stop Recognition", command=self.stop_recognition)
        self.btn_stop.pack()

        self.running = False  # Flag to control the recognition loop

    def browse_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg *.jpeg *.png")])
        if file_path:
            self.image = Image.open(file_path)
            self.image = self.image.resize((400, 300), Image.ANTIALIAS)
            self.photo = ImageTk.PhotoImage(self.image)
            self.label_image.config(image=self.photo)

    def start_recognition(self):
        self.running = True  # Set the flag to start the recognition loop
        while self.running:
            frames = pipeline.wait_for_frames()
            color_frame = frames.get_color_frame()
            depth_frame = frames.get_depth_frame()

            if not color_frame or not depth_frame:
                continue

            color_image = np.asanyarray(color_frame.get_data())
            depth_image = np.asanyarray(depth_frame.get_data())

            faces = detector(color_image)

            for face in faces:
                shape = predictor(color_image, face)
                face_descriptor_dlib = face_recognizer.compute_face_descriptor(color_image, shape)
                face_descriptor_numpy = np.array(face_descriptor_dlib)

                # Get 2D bounding box coordinates
                (x, y, w, h) = (face.left(), face.top(), face.width(), face.height())

                # Get person's geo coordinates based on camera's location
                person_geotags = get_person_geotags(color_frame, depth_frame, (x, y, w, h))

                # Compare the face descriptor with the search image
                match = face_recognition.compare_faces([search_face_descriptor], face_descriptor_numpy, tolerance=0.6)[0]

                if match and person_geotags:
                    # Draw bounding box and text on the color image
                    cv2.rectangle(color_image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    cv2.putText(color_image, f"Lat: {29.891293 + person_geotags[0]:.6f}, Long: {77.96101021 + person_geotags[1]:.6f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                    # Print or use the person's geo coordinates
                    print(f"Person Geotags: Latitude {29.891293 + person_geotags[0]:.6f}, Longitude {77.96101021 + person_geotags[1]:.6f}")

            # Display the color image with bounding boxes and coordinates
            cv2.imshow("RealSense Camera Search", color_image)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    def stop_recognition(self):
        self.running = False  # Set the flag to stop the recognition loop
        pipeline.stop()
        cv2.destroyAllWindows()

# Create an instance of the Tkinter window
root = tk.Tk()
app = App(root)
root.mainloop()


C:\Users\saini\AppData\Local\Temp\ipykernel_12296\479806330.py:63: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  self.image = self.image.resize((400, 300), Image.ANTIALIAS)


Person Geotags: Latitude 30.051648, Longitude 77.811555
Person Geotags: Latitude 30.034021, Longitude 77.811089
Person Geotags: Latitude 30.034465, Longitude 77.810622
Person Geotags: Latitude 30.034687, Longitude 77.810389
Person Geotags: Latitude 30.034465, Longitude 77.810622
Person Geotags: Latitude 30.034243, Longitude 77.810855
Person Geotags: Latitude 30.034021, Longitude 77.811089
Person Geotags: Latitude 30.034021, Longitude 77.811089
Person Geotags: Latitude 30.033577, Longitude 77.811555
Person Geotags: Latitude 30.032911, Longitude 77.812254
Person Geotags: Latitude 30.041485, Longitude 77.806823
Person Geotags: Latitude 30.041485, Longitude 77.806823
Person Geotags: Latitude 30.030913, Longitude 77.814353
Person Geotags: Latitude 30.054149, Longitude 77.809223
Person Geotags: Latitude 30.053399, Longitude 77.809923
Person Geotags: Latitude 30.053149, Longitude 77.810156
Person Geotags: Latitude 30.055386, Longitude 77.822618
Person Geotags: Latitude 30.089776, Longitude 77

In [ ]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import cv2
import dlib
import face_recognition
import numpy as np
import pyrealsense2 as rs
import threading

# Load pre-trained models
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks (2).dat")
face_recognizer = dlib.face_recognition_model_v1("dlib_face_recognition_resnet_model_v1.dat")

# Configure the RealSense camera
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)

# Start streaming
pipeline.start(config)

# Function to get person's geo coordinates based on camera's location
def get_person_geotags(color_frame, depth_frame, face_rect):
    # Calculate depth information for the center of the face bounding box
    depth_center = depth_frame.get_distance(int(face_rect[0] + face_rect[2] / 2), int(face_rect[1] + face_rect[3] / 2))

    # Get the exact 3D coordinates in meters
    depth_intrin = depth_frame.profile.as_video_stream_profile().intrinsics
    depth_point = rs.rs2_deproject_pixel_to_point(depth_intrin, [int(face_rect[0] + face_rect[2] / 2), int(face_rect[1] + face_rect[3] / 2)], depth_center)

    return depth_point

# Load a sample image to search for
search_image = face_recognition.load_image_file("C:\\Users\\saini\\Downloads\\WhatsApp Image 2024-02-05 at 9.17.11 PM.jpeg")
search_face_descriptor = face_recognition.face_encodings(search_image)[0]

# Create Tkinter GUI
class App:
    def __init__(self, window):
        self.window = window
        self.window.title("Face Recognition App")

        self.btn_browse = tk.Button(window, text="Browse Image", command=self.browse_image)
        self.btn_browse.pack()

        self.label_image = tk.Label(window)
        self.label_image.pack()

        self.btn_start = tk.Button(window, text="Start Recognition", command=self.start_recognition)
        self.btn_start.pack()

        self.btn_stop = tk.Button(window, text="Stop Recognition", command=self.stop_recognition)
        self.btn_stop.pack()

        self.running = False  # Flag to control the recognition loop
        self.thread = None  # Thread for running recognition

    def browse_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg *.jpeg *.png")])
        if file_path:
            self.image = Image.open(file_path)
            self.image = self.image.resize((400, 300), Image.ANTIALIAS)
            self.photo = ImageTk.PhotoImage(self.image)
            self.label_image.config(image=self.photo)

    def start_recognition(self):
        self.running = True
        self.thread = threading.Thread(target=self.recognition_thread)
        self.thread.start()

    def stop_recognition(self):
        self.running = False

    def recognition_thread(self):
        while self.running:
            frames = pipeline.wait_for_frames()
            color_frame = frames.get_color_frame()
            depth_frame = frames.get_depth_frame()

            if not color_frame or not depth_frame:
                continue

            color_image = np.asanyarray(color_frame.get_data())
            depth_image = np.asanyarray(depth_frame.get_data())

            faces = detector(color_image)

            for face in faces:
                shape = predictor(color_image, face)
                face_descriptor_dlib = face_recognizer.compute_face_descriptor(color_image, shape)
                face_descriptor_numpy = np.array(face_descriptor_dlib)

                # Get 2D bounding box coordinates
                (x, y, w, h) = (face.left(), face.top(), face.width(), face.height())

                # Get person's geo coordinates based on camera's location
                person_geotags = get_person_geotags(color_frame, depth_frame, (x, y, w, h))

                # Compare the face descriptor with the search image
                match = face_recognition.compare_faces([search_face_descriptor], face_descriptor_numpy, tolerance=0.6)[0]

                if match and person_geotags:
                    # Draw bounding box and text on the color image
                    cv2.rectangle(color_image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    cv2.putText(color_image, f"Lat: {29.891293 + person_geotags[0]:.6f}, Long: {77.96101021 + person_geotags[1]:.6f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                    # Print or use the person's geo coordinates
                    print(f"Person Geotags: Latitude {29.891293 + person_geotags[0]:.6f}, Longitude {77.96101021 + person_geotags[1]:.6f}")

            # Display the color image with bounding boxes and coordinates
            cv2.imshow("RealSense Camera Search", color_image)

            if cv2.waitKey(1) & 0xFF == ord('q') or not self.running:
                break

        pipeline.stop()
        cv2.destroyAllWindows()

# Create an instance of the Tkinter window
root = tk.Tk()
app = App(root)
root.mainloop()


C:\Users\saini\AppData\Local\Temp\ipykernel_22184\2624710322.py:65: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  self.image = self.image.resize((400, 300), Image.ANTIALIAS)


Person Geotags: Latitude 30.040861, Longitude 77.822353
Person Geotags: Latitude 30.040651, Longitude 77.822548
Person Geotags: Latitude 30.040861, Longitude 77.822353
Person Geotags: Latitude 30.041703, Longitude 77.821573
Person Geotags: Latitude 30.041492, Longitude 77.821768


Exception in thread Thread-6:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\saini\AppData\Local\Temp\ipykernel_22184\2624710322.py", line 79, in recognition_thread
RuntimeError: wait_for_frames cannot be called before start()
Exception in thread Thread-7:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\saini\AppData\Local\Temp\ipykernel_22184\2624710322.py", line 79, in recognition_thread
RuntimeError: wait_for_frames cannot be called before start()
Exception in thread Thread-8:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\threading.

In [ ]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import cv2
import dlib
import face_recognition
import numpy as np
import pyrealsense2 as rs
import threading

# Load pre-trained models
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks (2).dat")
face_recognizer = dlib.face_recognition_model_v1("dlib_face_recognition_resnet_model_v1.dat")

# Load a sample image to search for
search_image = face_recognition.load_image_file("C:\\Users\\saini\\Downloads\\WhatsApp Image 2024-02-05 at 9.17.11 PM.jpeg")
search_face_descriptor = face_recognition.face_encodings(search_image)[0]

# Create Tkinter GUI
class App:
    def __init__(self, window):
        self.window = window
        self.window.title("Face Recognition App")

        self.btn_browse = tk.Button(window, text="Browse Image", command=self.browse_image)
        self.btn_browse.pack()

        self.label_image = tk.Label(window)
        self.label_image.pack()

        self.btn_start = tk.Button(window, text="Start Recognition", command=self.start_recognition)
        self.btn_start.pack()

        self.btn_stop = tk.Button(window, text="Stop Recognition", command=self.stop_recognition)
        self.btn_stop.pack()

        self.running = False  # Flag to control the recognition loop
        self.thread = None  # Thread for running recognition
        self.pipeline = None  # RealSense pipeline

    def browse_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg *.jpeg *.png")])
        if file_path:
            self.image = Image.open(file_path)
            self.image = self.image.resize((400, 300), Image.LANCZOS)  # Use LANCZOS resampling method
            self.photo = ImageTk.PhotoImage(self.image)
            self.label_image.config(image=self.photo)

    def start_recognition(self):
        if not self.running:
            self.running = True
            self.pipeline = rs.pipeline()
            config = rs.config()
            config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
            config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
            self.pipeline.start(config)
            self.thread = threading.Thread(target=self.recognition_thread)
            self.thread.start()

    def stop_recognition(self):
        self.running = False
        if self.pipeline and self.pipeline.is_started():
            self.pipeline.stop()

    def recognition_thread(self):
        while self.running:
            frames = self.pipeline.wait_for_frames()
            color_frame = frames.get_color_frame()
            depth_frame = frames.get_depth_frame()

            if not color_frame or not depth_frame:
                continue

            color_image = np.asanyarray(color_frame.get_data())
            depth_image = np.asanyarray(depth_frame.get_data())

            faces = detector(color_image)

            for face in faces:
                shape = predictor(color_image, face)
                face_descriptor_dlib = face_recognizer.compute_face_descriptor(color_image, shape)
                face_descriptor_numpy = np.array(face_descriptor_dlib)

                # Get 2D bounding box coordinates
                (x, y, w, h) = (face.left(), face.top(), face.width(), face.height())

                # Compare the face descriptor with the search image
                match = face_recognition.compare_faces([search_face_descriptor], face_descriptor_numpy, tolerance=0.6)[0]

                if match:
                    # Draw bounding box and text on the color image
                    cv2.rectangle(color_image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    cv2.putText(color_image, "Match Found", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # Display the color image with bounding boxes and coordinates
            cv2.imshow("RealSense Camera Search", color_image)

            if cv2.waitKey(1) & 0xFF == ord('q') or not self.running:
                break

        if self.pipeline and self.pipeline.is_started():
            self.pipeline.stop()
        cv2.destroyAllWindows()

# Create an instance of the Tkinter window
root = tk.Tk()
app = App(root)
root.mainloop()


In [1]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import cv2
import dlib
import face_recognition
import numpy as np
import pyrealsense2 as rs
import threading

# Load pre-trained models
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks (2).dat")
face_recognizer = dlib.face_recognition_model_v1("dlib_face_recognition_resnet_model_v1.dat")

# Configure the RealSense camera
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)

# Start streaming
pipeline.start(config)

# Function to get person's geo coordinates based on camera's location
def get_person_geotags(color_frame, depth_frame, face_rect):
    # Calculate depth information for the center of the face bounding box
    depth_center = depth_frame.get_distance(int(face_rect[0] + face_rect[2] / 2), int(face_rect[1] + face_rect[3] / 2))

    # Get the exact 3D coordinates in meters
    depth_intrin = depth_frame.profile.as_video_stream_profile().intrinsics
    depth_point = rs.rs2_deproject_pixel_to_point(depth_intrin, [int(face_rect[0] + face_rect[2] / 2), int(face_rect[1] + face_rect[3] / 2)], depth_center)

    return depth_point

# Load a sample image to search for
search_image = face_recognition.load_image_file("C:\\Users\\saini\\Downloads\\WhatsApp Image 2024-02-05 at 9.17.11 PM.jpeg")
search_face_descriptor = face_recognition.face_encodings(search_image)[0]

# Create Tkinter GUI
class App:
    def _init_(self, window):
        self.window = window
        self.window.title("Face Recognition App")

        self.btn_browse = tk.Button(window, text="Browse Image", command=self.browse_image)
        self.btn_browse.pack()

        self.label_image = tk.Label(window)
        self.label_image.pack()

        self.btn_start = tk.Button(window, text="Start Recognition", command=self.start_recognition)
        self.btn_start.pack()

        self.btn_stop = tk.Button(window, text="Stop Recognition", command=self.stop_recognition)
        self.btn_stop.pack()

        self.running = False  # Flag to control the recognition loop
        self.thread = None  # Thread for running recognition

    def browse_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg *.jpeg *.png")])
        if file_path:
            self.image = Image.open(file_path)
            self.image = self.image.resize((400, 300), Image.ANTIALIAS)
            self.photo = ImageTk.PhotoImage(self.image)
            self.label_image.config(image=self.photo)

    def start_recognition(self):
        self.running = True
        self.thread = threading.Thread(target=self.recognition_thread)
        self.thread.start()

    def stop_recognition(self):
        self.running = False

    def recognition_thread(self):
        while self.running:
            frames = pipeline.wait_for_frames()
            color_frame = frames.get_color_frame()
            depth_frame = frames.get_depth_frame()

            if not color_frame or not depth_frame:
                continue

            color_image = np.asanyarray(color_frame.get_data())
            depth_image = np.asanyarray(depth_frame.get_data())

            faces = detector(color_image)

            for face in faces:
                shape = predictor(color_image, face)
                face_descriptor_dlib = face_recognizer.compute_face_descriptor(color_image, shape)
                face_descriptor_numpy = np.array(face_descriptor_dlib)

                # Get 2D bounding box coordinates
                (x, y, w, h) = (face.left(), face.top(), face.width(), face.height())

                # Get person's geo coordinates based on camera's location
                person_geotags = get_person_geotags(color_frame, depth_frame, (x, y, w, h))

                # Compare the face descriptor with the search image
                match = face_recognition.compare_faces([search_face_descriptor], face_descriptor_numpy, tolerance=0.6)[0]

                if match and person_geotags:
                    # Draw bounding box and text on the color image
                    cv2.rectangle(color_image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    cv2.putText(color_image, f"Lat: {29.891293 + person_geotags[0]:.6f}, Long: {77.96101021 + person_geotags[1]:.6f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                    # Print or use the person's geo coordinates
                    print(f"Person Geotags: Latitude {29.891293 + person_geotags[0]:.6f}, Longitude {77.96101021 + person_geotags[1]:.6f}")

            # Display the color image with bounding boxes and coordinates
            cv2.imshow("RealSense Camera Search", color_image)

            if cv2.waitKey(1) & 0xFF == ord('q') or not self.running:
                break

        pipeline.stop()
        cv2.destroyAllWindows()

# Create an instance of the Tkinter window
root = tk.Tk()
app = App(root)
root.mainloop()

TypeError: App() takes no arguments